In [41]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
print(df.head(5))

      id  gender age  hypertension  heart_disease ever_married work_type  \
0   1192  Female  31             0              0           No  Govt_job   
1     77  Female  13             0              0           No  children   
2  59200    Male  18             0              0           No   Private   
3  24905  Female  65             0              0          Yes   Private   
4  24257    Male   4             0              0           No  children   

  Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0          Rural              70.66  27.2     never smoked       0  
1          Rural              85.81  18.6          Unknown       0  
2          Urban              60.56  33.0     never smoked       0  
3          Urban             205.77  46.0  formerly smoked       1  
4          Rural              90.42  16.2          Unknown       0  


:::{.callout-important}
bmi컬럼의 각 결측치들을 직전의 행의 bmi값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라([출처](https://cafe.naver.com/yjbooks/63744?boardType=L))
:::

In [42]:
print('{:.3f}'.format(df['bmi'].fillna(method='ffill').mean()))

29.188


:::{.callout-important}
bim컬럼의 각 결측치를 결측치를 가진 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 데체된 bmi 컬럼의 평균을 소숫점 이하 3자리까지 구하여라([출처](https://cafe.naver.com/yjbooks/63744?boardType=L))
:::

1. 나이를 구간으로 분류하기 -> `np.digitize(변수, bins=[경계점])` eg. df['age_category'] = np.digitize(df['age'], bins=[10, 20, 30, 40, 50, 60, 70, 80])
1. 구간으로 groupby 하여 평균을 구한 후 딕셔너리형으로 만들기 -> age_bmi = {i:v for i, v in enumerate(df.groupby('age_category').mean()['bmi'])}
1. bmi 컬럼 결측치 슬라이싱 후 'age_category' 열만 뽑은 후 map으로 나이대별 평균값 뽑기 -> df.loc[df['bmi'].isna(), 'age_category'].map(age_bmi)
1. 나이대별 평균값을 bmi 결측치에 대입 -> df.loc[df['bmi'].isna(), 'bmi'] = df.loc[df['bmi'].isna(), 'age_category'].map(age_bmi)

In [66]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')

# age 특성에 특수 문자 포함된 값이 있어 수치형으로 변환이 안 됨
df['age'].unique()

array(['31', '13', '18', '65', '4', '28', '64', '62', '26', '63', '50',
       '77', '1', '52', '24', '48', '45', '74', '3', '17', '54', '55',
       '67', '35', '38', '81', '34', '44', '79', '56', '70', '30', '39',
       '29', '80', '59', '51', '19', '43', '71', '0', '23', '53', '78',
       '66', '60', '76', '22', '6', '47', '2', '46', '11', '33', '37',
       '49', '61', '75', '40', '8', '57', '12', '21', '41', '58', '27',
       '20', '68', '42', '5', '69', '9', '36', '25', '82', '73', '32',
       '7', '16', '14', '72', '15', '10', '*82'], dtype=object)

In [67]:
# 특수문자 제거하고 수치형으로 변환
df['age'] = df['age'].replace('*82', '82')
df['age'] = df['age'].astype('int')
df['age'].unique()

array([31, 13, 18, 65,  4, 28, 64, 62, 26, 63, 50, 77,  1, 52, 24, 48, 45,
       74,  3, 17, 54, 55, 67, 35, 38, 81, 34, 44, 79, 56, 70, 30, 39, 29,
       80, 59, 51, 19, 43, 71,  0, 23, 53, 78, 66, 60, 76, 22,  6, 47,  2,
       46, 11, 33, 37, 49, 61, 75, 40,  8, 57, 12, 21, 41, 58, 27, 20, 68,
       42,  5, 69,  9, 36, 25, 82, 73, 32,  7, 16, 14, 72, 15, 10])

In [68]:
# 최대 최솟값 확인
df[['age','bmi']].describe()

,age,bmi
count,1137.000000,1085.000000
mean,45.189094,29.198065
std,23.070207,7.669615
min,0.000000,11.300000
25%,28.000000,24.100000
50%,48.000000,28.500000
75%,63.000000,33.200000
max,82.000000,64.400000


In [69]:
# 나이를 10단위로 구분하기
df['age_category'] = np.digitize(df['age'], bins=[10, 20, 30, 40, 50, 60, 70, 80])
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_category
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0,3
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0,1
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0,1
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1,6
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0,0


In [70]:
# 나이대별 bmi 평균 -> 딕셔너리로
age_bmi = { i:v for i, v in enumerate(df.groupby('age_category').mean()['bmi'])}
age_bmi

{0: 18.96078431372549,
 1: 24.477227722772277,
 2: 28.052747252747256,
 3: 31.751111111111108,
 4: 31.427096774193547,
 5: 31.894086021505377,
 6: 31.615873015873014,
 7: 30.44772727272727,
 8: 28.566666666666666}

In [72]:
# bmi 결측치를 나이대별 bmi 평균으로 채우기
df.loc[df['bmi'].isna(), 'bmi'] = df.loc[df['bmi'].isna(), 'age_category'].map(age_bmi)
df['bmi'].isna().sum()

0

In [73]:
print('{:.3f}'.format(df['bmi'].mean()))

29.263
